In [299]:
%matplotlib inline
# Dependencies and Setup
import pandas as pd
import numpy as np
from datetime import datetime
import dill
import requests
from bs4 import BeautifulSoup
import html
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from collections import defaultdict

- Open html files saved from https://www.federalreserve.gov/newsevents/speeches.htm, selected pages displaying Jerome Powell's speeches, 3 total.
- Deploy BeautifulSoup to scrape needed data from the files.

In [300]:
links=[]

In [301]:
f = open (".\data\powell_p1.html", encoding='utf8')
soup = BeautifulSoup(f)
f.close()
print(soup.prettify())

<html class="js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths grunticon" lang="en" style="">
 <head>
  <style type="text/css">
   [uib-typeahead-popup].dropdown-menu{display:block;}
  </style>
  <style type="text/css">
   .uib-time input{width:50px;}
  </style>
  <style type="text/css">
   [uib-tooltip-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-

In [302]:
links_p1 = soup.find_all('a', class_='ng-binding')
links_p1
for a in links_p1:
#     print(a['href'])
    if a['href'] != '':
        links.append(a['href'])

In [303]:
f = open (".\data\powell_p2.html", encoding='utf8')
soup = BeautifulSoup(f)
f.close()
print(soup.prettify())

<html class="js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths grunticon" lang="en" style="">
 <head>
  <style type="text/css">
   [uib-typeahead-popup].dropdown-menu{display:block;}
  </style>
  <style type="text/css">
   .uib-time input{width:50px;}
  </style>
  <style type="text/css">
   [uib-tooltip-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-

In [304]:
links_p2 = soup.find_all('a', class_='ng-binding')
links_p2
for a in links_p2:
#     print(a['href'])
    if a['href'] != '':
        links.append(a['href'])

In [305]:
f = open (".\data\powell_p3.html", encoding='utf8')
soup = BeautifulSoup(f)
f.close()
print(soup.prettify())

<html class="js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths grunticon" lang="en" style="">
 <head>
  <style type="text/css">
   [uib-typeahead-popup].dropdown-menu{display:block;}
  </style>
  <style type="text/css">
   .uib-time input{width:50px;}
  </style>
  <style type="text/css">
   [uib-tooltip-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-

In [306]:
links_p3 = soup.find_all('a', class_='ng-binding')
links_p3
for a in links_p3:
#     print(a['href'])
    if a['href'] != '':
        links.append(a['href'])

Extract from the above links to create a dataframe with columns of "date" and "text".

In [307]:
data=[]
for url in links:
    page = requests.get(f"https://www.federalreserve.gov{url}")
    soup2 = BeautifulSoup(page.content, 'html.parser', from_encoding="UTF-8")
    mydict={}
    title = soup2.find_all('h3', class_='title')
    mydict['title']=title[0].getText()
    date = soup2.find_all('p', class_='article__time')
    mydict['date']=date[0].getText()
    location = soup2.find_all('p', class_='location')
    mydict['location']=location[0].getText()
    text = soup2.find('div', id='article').find_all('div')
    for a in text:
        if len(a.getText())>400:
            text=''
            for p in a.find_all('p'):
                if len(p.find_all('a')) == 0:
                    text+=p.getText()

    mydict['text']=text
    data.append(mydict)
df = pd.DataFrame(data)
df = df.loc[:, ["date", "text"]]
df

,date,text
0,"March 18, 2021",I would like to thank Sir Jon Cunliffe and the...
1,"February 10, 2021",Today I will discuss the state of our labor ma...
2,"October 06, 2020",Good morning. It has been just eight months si...
3,"August 27, 2020","Thank you, Esther, for that introduction, and ..."
4,"June 19, 2020","Thank you, President Mester and Treye Johnson,..."
5,"May 21, 2020",Good afternoon. I just want to say a few words...
6,"May 13, 2020",The coronavirus has left a devastating human a...
7,"April 09, 2020",Good morning. The challenge we face today is d...
8,"November 25, 2019","Over the past year, my colleagues and I on the..."
9,"October 09, 2019",Good morning. I am happy to be here in Kansas ...


In [308]:
df['date']= pd.to_datetime(df['date'])

In [309]:
df.style.format({"date": lambda t: t.strftime("%Y-%m-%d")})
df.head(2)    

,date,text
0,2021-03-18,I would like to thank Sir Jon Cunliffe and the...
1,2021-02-10,Today I will discuss the state of our labor ma...


In [310]:
# Save the dataframe to a csv file for web display usage
df.to_csv('./Data/powell_sp.csv', index=False)

In [311]:
# Initiate economic keywords from Powell's speeches
speeches = df.text

In [312]:
len(speeches)

60

Deploy CountVectorier to find the top most important keywords in one or two words. 

In [313]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(speeches)
names = vectorizer.get_feature_names()
names

['000',
 '03',
 '10',
 '100',
 '101',
 '102',
 '1029',
 '104',
 '10580',
 '107',
 '108',
 '109',
 '11',
 '1109',
 '113',
 '116',
 '119',
 '11th',
 '12',
 '121',
 '125',
 '128',
 '128th',
 '129',
 '12th',
 '13',
 '138',
 '13th',
 '14',
 '1483',
 '15',
 '1525',
 '1553',
 '1557',
 '157',
 '16',
 '1687',
 '1692',
 '17',
 '170',
 '1720',
 '1726',
 '1870',
 '19',
 '1913',
 '1930s',
 '1934',
 '1935',
 '1937',
 '1938',
 '1940',
 '1940s',
 '1944',
 '195',
 '1950',
 '1950s',
 '1951',
 '1959',
 '1960',
 '1960s',
 '1965',
 '1966',
 '1967',
 '1969',
 '1970',
 '1970s',
 '1977',
 '1980s',
 '1981',
 '1982',
 '1983',
 '1985',
 '1987',
 '1990s',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '19th',
 '20',
 '200',
 '2000',
 '2000s',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2023',
 '2066',
 '20th',
 '21',
 '210',
 '214',
 

In [314]:
sums_x = x.sum(axis=0)
sums_x

matrix([[ 7,  1, 16, ...,  2,  1,  1]], dtype=int64)

In [315]:
singlelist={}
single ={}

for x in range(len(names)):
    mydict={}
    mydict['count']=sums_x.tolist()[0][x]
    mydict['prob']=sums_x.tolist()[0][x]/60
    singlelist[names[x]]=mydict
singlelist

{'000': {'count': 7, 'prob': 0.11666666666666667},
 '03': {'count': 1, 'prob': 0.016666666666666666},
 '10': {'count': 16, 'prob': 0.26666666666666666},
 '100': {'count': 7, 'prob': 0.11666666666666667},
 '101': {'count': 1, 'prob': 0.016666666666666666},
 '102': {'count': 2, 'prob': 0.03333333333333333},
 '1029': {'count': 1, 'prob': 0.016666666666666666},
 '104': {'count': 1, 'prob': 0.016666666666666666},
 '10580': {'count': 1, 'prob': 0.016666666666666666},
 '107': {'count': 1, 'prob': 0.016666666666666666},
 '108': {'count': 2, 'prob': 0.03333333333333333},
 '109': {'count': 1, 'prob': 0.016666666666666666},
 '11': {'count': 7, 'prob': 0.11666666666666667},
 '1109': {'count': 1, 'prob': 0.016666666666666666},
 '113': {'count': 1, 'prob': 0.016666666666666666},
 '116': {'count': 2, 'prob': 0.03333333333333333},
 '119': {'count': 1, 'prob': 0.016666666666666666},
 '11th': {'count': 5, 'prob': 0.08333333333333333},
 '12': {'count': 21, 'prob': 0.35},
 '121': {'count': 1, 'prob': 0.01

In [316]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2,2), min_df=15)
y = vectorizer2.fit_transform(speeches)
combos = vectorizer2.get_feature_names()
sums_y = y.sum(axis = 0)
sums_y
combos

['able to',
 'about the',
 'all of',
 'and financial',
 'and in',
 'and inflation',
 'and other',
 'and that',
 'and the',
 'and to',
 'and we',
 'are not',
 'around the',
 'as the',
 'as we',
 'as well',
 'at the',
 'banks and',
 'before the',
 'businesses and',
 'but the',
 'but we',
 'by the',
 'can be',
 'central banks',
 'changes in',
 'close to',
 'colleagues and',
 'continue to',
 'crisis and',
 'do not',
 'during the',
 'economic growth',
 'economy and',
 'economy is',
 'employment and',
 'federal funds',
 'federal open',
 'federal reserve',
 'financial crisis',
 'financial stability',
 'financial system',
 'for example',
 'for the',
 'forward to',
 'from the',
 'funds rate',
 'growth and',
 'has been',
 'have been',
 'if the',
 'importance of',
 'in addition',
 'in our',
 'in the',
 'in this',
 'interest rates',
 'is not',
 'is now',
 'is that',
 'is the',
 'is to',
 'it is',
 'labor market',
 'level of',
 'likely to',
 'longer term',
 'look forward',
 'low and',
 'many of',
 

In [317]:
combolist={}
addvalue=5

for x in range(len(combos)):
    mydict={}
    mydict['count']=sums_y.tolist()[0][x]
    mydict['prob']=sums_y.tolist()[0][x]/60
    mydict['w1'] = combos[x].split()[0]
    mydict['w2'] = combos[x].split()[1]
    mydict['num'] = (sums_y.tolist()[0][x]/60)
    mydict['w1denom'] = (singlelist[mydict['w1']]['count'] + addvalue)/(60 + addvalue)
    mydict['w2denom'] = (singlelist[mydict['w2']]['count'] + addvalue)/(60 + addvalue)
    mydict['importance'] = mydict['num'] /(mydict['w1denom'] * mydict['w2denom'])
    combolist[combos[x]]=mydict
     

combo100 = sorted(combolist.items(),key=lambda x: x[1]['importance'], reverse=True)
combo100[:100]
[x[0] for x in combo100[:100]]

['united states',
 'maximum employment',
 'look forward',
 'short term',
 'thank you',
 'my colleagues',
 'interest rates',
 'longer term',
 'price stability',
 'federal reserve',
 'funds rate',
 'federal funds',
 'percent objective',
 'monetary policy',
 'labor market',
 'open market',
 'over time',
 'federal open',
 'financial stability',
 'central banks',
 'do not',
 'has been',
 'financial system',
 'such as',
 'for example',
 'have been',
 'market committee',
 'financial crisis',
 'will continue',
 'in addition',
 'may be',
 'economic growth',
 'can be',
 'there is',
 'as well',
 'there are',
 'today will',
 'it is',
 'will be',
 'able to',
 'well as',
 'want to',
 'changes in',
 'of course',
 'to ensure',
 'continue to',
 'we are',
 'close to',
 'opportunity to',
 'we have',
 'range of',
 'importance of',
 'to address',
 'is now',
 'to speak',
 'colleagues and',
 'to say',
 'part of',
 'understanding of',
 'to provide',
 'we can',
 'you for',
 'need to',
 'likely to',
 'level of'

In [318]:
combolist

{'able to': {'count': 19,
  'prob': 0.31666666666666665,
  'w1': 'able',
  'w2': 'to',
  'num': 0.31666666666666665,
  'w1denom': 0.38461538461538464,
  'w2denom': 32.30769230769231,
  'importance': 0.02548412698412698},
 'about the': {'count': 41,
  'prob': 0.6833333333333333,
  'w1': 'about',
  'w2': 'the',
  'num': 0.6833333333333333,
  'w1denom': 2.353846153846154,
  'w2denom': 64.18461538461538,
  'importance': 0.004522968769909157},
 'all of': {'count': 35,
  'prob': 0.5833333333333334,
  'w1': 'all',
  'w2': 'of',
  'num': 0.5833333333333334,
  'w1denom': 2.0,
  'w2denom': 32.50769230769231,
  'importance': 0.00897223536835463},
 'and financial': {'count': 30,
  'prob': 0.5,
  'w1': 'and',
  'w2': 'financial',
  'num': 0.5,
  'w1denom': 36.13846153846154,
  'w2denom': 4.753846153846154,
  'importance': 0.002910417019705417},
 'and in': {'count': 28,
  'prob': 0.4666666666666667,
  'w1': 'and',
  'w2': 'in',
  'num': 0.4666666666666667,
  'w1denom': 36.13846153846154,
  'w2denom'

In [319]:
from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS #.union(my_additional_stop_words)
vectorizer = CountVectorizer(lowercase=True, stop_words=stop_words)

vector_text = speeches

cv_fit = vectorizer.fit_transform(vector_text)

In [320]:
words = vectorizer.vocabulary_

In [321]:
feature_names = vectorizer.get_feature_names()
feature_counts = cv_fit.toarray().sum(axis=0)

In [322]:
final_count_df = pd.DataFrame({"word": feature_names, "cnt_word": feature_counts})
final_count_df.sort_values(by='cnt_word', ascending=False)

,word,cnt_word
3598,policy,350
2564,inflation,329
2022,financial,304
1628,economy,250
3031,market,225
...,...,...
3849,pushed,1
2943,longtime,1
3851,puts,1
1521,distinctly,1


In [323]:
final_count_df.to_csv('./Data/powell_sp_count.csv', index=False)

In [324]:
# Introduce adjusted economic keywords extracted from top 100 words in the above combolist initiated in Powell's speeches + keywords in economic indicators
trade_matchers = ["bankruptcies",
"broadband internet penetration",
"building permit",
"business cycle",
"central banks",
"changes in",
"china",
"construction spending",
"consumer confidence index",
"consumer leverage ratio",
"consumer price index",
"consumer spending",
"during the",
"economic growth",
"employment",
"export",
"federal funds",
"federal open",
"federal reserve",
"financial crisis",
"financial stability",
"financial system",
"forward to",
"funds rate",
"gdp",
"home building",
"housing starts",
"import",
"importance of",
"industrial production",
"inflation",
"interest rate spread",
"interest rates",
"jobless claims",
"labor market",
"level of",
"likely to",
"longer term",
"look forward",
"manufacture new order",
"manufacture",
"manufacturing demand",
"manufacturing",
"market committee",
"market",
"maximum employment",
"monetary policy",
"money supply",
"money",
"monthly job added",
"NAFTA",
"NASDAQ",
"need to",
"new residential sales",
"oil",
"open market",
"opportunity to",
"our percent",
"over time",
"percent objective",
"prime rate",
"producer price index",
"quarterly change in gdp",
"retail sales",
"review of",
"S&P 500",
"short term",
"stock market prices",
"stock market",
"stocks",
"tariff",
"tax",
"the committee",
"the economy",
"the fed",
"the federal",
"the fomc",
"the great",
"the outlook",
"the united",
"the world",
"to address",
"to continue",
"to ensure",
"to help",
"to make",
"to provide",
"to speak",
"to support",
"trade",
"turn to",
"understanding of",
"unemployment",
"united states",
"vendor performance",
"weekly hours",
"wholesale price index"]

In [325]:
df['week'] = (df.date - df['date'].dt.weekday * np.timedelta64(1,'D'))
df['week'] = pd.DatetimeIndex(df['week']).normalize()
df['day'] = pd.DatetimeIndex(df['date']).normalize()
df

,date,text,week,day
0,2021-03-18,I would like to thank Sir Jon Cunliffe and the...,2021-03-15,2021-03-18
1,2021-02-10,Today I will discuss the state of our labor ma...,2021-02-08,2021-02-10
2,2020-10-06,Good morning. It has been just eight months si...,2020-10-05,2020-10-06
3,2020-08-27,"Thank you, Esther, for that introduction, and ...",2020-08-24,2020-08-27
4,2020-06-19,"Thank you, President Mester and Treye Johnson,...",2020-06-15,2020-06-19
5,2020-05-21,Good afternoon. I just want to say a few words...,2020-05-18,2020-05-21
6,2020-05-13,The coronavirus has left a devastating human a...,2020-05-11,2020-05-13
7,2020-04-09,Good morning. The challenge we face today is d...,2020-04-06,2020-04-09
8,2019-11-25,"Over the past year, my colleagues and I on the...",2019-11-25,2019-11-25
9,2019-10-09,Good morning. I am happy to be here in Kansas ...,2019-10-07,2019-10-09


In [326]:
# Count frequencies of economic keywords by week
countbyweek=defaultdict(int)
for index,row in df.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in trade_matchers:
            countbyweek[row['week']] += 1
#     break
countbyweek

defaultdict(int,
            {Timestamp('2021-02-08 00:00:00'): 11,
             Timestamp('2020-10-05 00:00:00'): 21,
             Timestamp('2020-08-24 00:00:00'): 24,
             Timestamp('2020-06-15 00:00:00'): 1,
             Timestamp('2020-05-11 00:00:00'): 1,
             Timestamp('2020-04-06 00:00:00'): 3,
             Timestamp('2019-11-25 00:00:00'): 25,
             Timestamp('2019-10-07 00:00:00'): 42,
             Timestamp('2019-09-30 00:00:00'): 5,
             Timestamp('2019-08-19 00:00:00'): 26,
             Timestamp('2019-07-15 00:00:00'): 28,
             Timestamp('2019-06-24 00:00:00'): 13,
             Timestamp('2019-06-03 00:00:00'): 11,
             Timestamp('2019-05-20 00:00:00'): 6,
             Timestamp('2019-05-06 00:00:00'): 1,
             Timestamp('2019-03-04 00:00:00'): 8,
             Timestamp('2019-02-25 00:00:00'): 4,
             Timestamp('2019-02-11 00:00:00'): 2,
             Timestamp('2019-02-04 00:00:00'): 1,
             Timestamp('

In [327]:
# Count frequencies of economic keywords by day
countbyday=defaultdict(int)
for index,row in df.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in trade_matchers:
            countbyday[row['day']] += 1
#     break
countbyday

defaultdict(int,
            {Timestamp('2021-02-10 00:00:00'): 11,
             Timestamp('2020-10-06 00:00:00'): 21,
             Timestamp('2020-08-27 00:00:00'): 24,
             Timestamp('2020-06-19 00:00:00'): 1,
             Timestamp('2020-05-13 00:00:00'): 1,
             Timestamp('2020-04-09 00:00:00'): 3,
             Timestamp('2019-11-25 00:00:00'): 25,
             Timestamp('2019-10-09 00:00:00'): 5,
             Timestamp('2019-10-08 00:00:00'): 35,
             Timestamp('2019-10-07 00:00:00'): 2,
             Timestamp('2019-10-04 00:00:00'): 5,
             Timestamp('2019-08-23 00:00:00'): 26,
             Timestamp('2019-07-16 00:00:00'): 28,
             Timestamp('2019-06-25 00:00:00'): 13,
             Timestamp('2019-06-04 00:00:00'): 11,
             Timestamp('2019-05-20 00:00:00'): 6,
             Timestamp('2019-05-09 00:00:00'): 1,
             Timestamp('2019-03-08 00:00:00'): 8,
             Timestamp('2019-02-28 00:00:00'): 4,
             Timestamp('

In [328]:
# Create a dataframe of the frequency counts of economic keywords by week
powell_weekdf = pd.DataFrame([countbyweek]).transpose().reset_index()
powell_weekdf.columns=(['Week','Count'])
powell_weekdf

,Week,Count
0,2021-02-08,11
1,2020-10-05,21
2,2020-08-24,24
3,2020-06-15,1
4,2020-05-11,1
5,2020-04-06,3
6,2019-11-25,25
7,2019-10-07,42
8,2019-09-30,5
9,2019-08-19,26


In [329]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
powell_weekdf.to_csv('./Data/powell_weekdf_count.csv', index=False)

In [330]:
# Create a dataframe of the frequency counts of economic keywords by day
powell_daydf = pd.DataFrame([countbyday]).transpose().reset_index()
powell_daydf.columns=(['Date','Count'])
powell_daydf

,Date,Count
0,2021-02-10,11
1,2020-10-06,21
2,2020-08-27,24
3,2020-06-19,1
4,2020-05-13,1
5,2020-04-09,3
6,2019-11-25,25
7,2019-10-09,5
8,2019-10-08,35
9,2019-10-07,2


In [331]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
powell_daydf.to_csv('./Data/powell_daydf_count.csv', index=False)

In [332]:
# Introduce negative financial keywords
Negative = ['abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonments', 'abandons', 'abdicated',
                       'abdicates', 'abdicating', 'abdication', 'abdications', 'aberrant', 'aberration', 'aberrational',
                       'aberrations', 'abetting', 'abnormal', 'abnormalities', 'abnormality', 'abnormally', 'abolish',
                       'abolished', 'abolishes', 'abolishing', 'abrogate', 'abrogated', 'abrogates', 'abrogating',
                       'abrogation', 'abrogations', 'abrupt', 'abruptly', 'abruptness', 'absence', 'absences',
                       'absenteeism', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abusively', 'abusiveness',
                       'accident', 'accidental', 'accidentally', 'accidents', 'accusation', 'accusations', 'accuse',
                       'accused', 'accuses', 'accusing', 'acquiesce', 'acquiesced', 'acquiesces', 'acquiescing',
                       'acquit', 'acquits', 'acquittal', 'acquittals', 'acquitted', 'acquitting', 'adulterate',
                       'adulterated', 'adulterating', 'adulteration', 'adulterations', 'adversarial', 'adversaries',
                       'adversary', 'adverse', 'adversely', 'adversities', 'adversity', 'aftermath', 'aftermaths',
                       'against', 'aggravate', 'aggravated', 'aggravates', 'aggravating', 'aggravation', 'aggravations',
                       'alerted', 'alerting', 'alienate', 'alienated', 'alienates', 'alienating', 'alienation',
                       'alienations', 'allegation', 'allegations', 'allege', 'alleged', 'allegedly', 'alleges',
                       'alleging', 'annoy', 'annoyance', 'annoyances', 'annoyed', 'annoying', 'annoys', 'annul',
                       'annulled', 'annulling', 'annulment', 'annulments', 'annuls', 'anomalies', 'anomalous',
                       'anomalously', 'anomaly', 'anticompetitive', 'antitrust', 'argue', 'argued', 'arguing',
                       'argument', 'argumentative', 'arguments', 'arrearage', 'arrearages', 'arrears', 'arrest',
                       'arrested', 'arrests', 'artificially', 'assault', 'assaulted', 'assaulting', 'assaults',
                       'assertions', 'attrition', 'aversely', 'backdating', 'bad', 'bail', 'bailout', 'balk', 'balked',
                       'bankrupt', 'bankruptcies', 'bankruptcy', 'bankrupted', 'bankrupting', 'bankrupts', 'bans',
                       'barred', 'barrier', 'barriers', 'bottleneck', 'bottlenecks', 'boycott', 'boycotted',
                       'boycotting', 'boycotts', 'breach', 'breached', 'breaches', 'breaching', 'break', 'breakage',
                       'breakages', 'breakdown', 'breakdowns', 'breaking', 'breaks', 'bribe', 'bribed', 'briberies',
                       'bribery', 'bribes', 'bribing', 'bridge', 'broken', 'burden', 'burdened', 'burdening', 'burdens',
                       'burdensome', 'burned', 'calamities', 'calamitous', 'calamity', 'cancel', 'canceled',
                       'canceling', 'cancellation', 'cancellations', 'cancelled', 'cancelling', 'cancels', 'careless',
                       'carelessly', 'carelessness', 'catastrophe', 'catastrophes', 'catastrophic', 'catastrophically',
                       'caution', 'cautionary', 'cautioned', 'cautioning', 'cautions', 'cease', 'ceased', 'ceases',
                       'ceasing', 'censure', 'censured', 'censures', 'censuring', 'challenge', 'challenged',
                       'challenges', 'challenging', 'chargeoffs', 'circumvent', 'circumvented', 'circumventing',
                       'circumvention', 'circumventions', 'circumvents', 'claiming', 'claims', 'clawback', 'closed',
                       'closeout', 'closeouts', 'closing', 'closings', 'closure', 'closures', 'coerce', 'coerced',
                       'coerces', 'coercing', 'coercion', 'coercive', 'collapse', 'collapsed', 'collapses',
                       'collapsing', 'collision', 'collisions', 'collude', 'colluded', 'colludes', 'colluding',
                       'collusion', 'collusions', 'collusive', 'complain', 'complained', 'complaining', 'complains',
                       'complaint', 'complaints', 'complicate', 'complicated', 'complicates', 'complicating',
                       'complication', 'complications', 'compulsion', 'concealed', 'concealing', 'concede', 'conceded',
                       'concedes', 'conceding', 'concern', 'concerned', 'concerns', 'conciliating', 'conciliation',
                       'conciliations', 'condemn', 'condemnation', 'condemnations', 'condemned', 'condemning',
                       'condemns', 'condone', 'condoned', 'confess', 'confessed', 'confesses', 'confessing',
                       'confession', 'confine', 'confined', 'confinement', 'confinements', 'confines', 'confining',
                       'confiscate', 'confiscated', 'confiscates', 'confiscating', 'confiscation', 'confiscations',
                       'conflict', 'conflicted', 'conflicting', 'conflicts', 'confront', 'confrontation',
                       'confrontational', 'confrontations', 'confronted', 'confronting', 'confronts', 'confuse',
                       'confused', 'confuses', 'confusing', 'confusingly', 'confusion', 'conspiracies', 'conspiracy',
                       'conspirator', 'conspiratorial', 'conspirators', 'conspire', 'conspired', 'conspires',
                       'conspiring', 'contempt', 'contend', 'contended', 'contending', 'contends', 'contention',
                       'contentions', 'contentious', 'contentiously', 'contested', 'contesting', 'contraction',
                       'contractions', 'contradict', 'contradicted', 'contradicting', 'contradiction', 'contradictions',
                       'contradictory', 'contradicts', 'contrary', 'controversial', 'controversies', 'controversy',
                       'convict', 'convicted', 'convicting', 'conviction', 'convictions', 'corrected', 'correcting',
                       'correction', 'corrections', 'corrects', 'corrupt', 'corrupted', 'corrupting', 'corruption',
                       'corruptions', 'corruptly', 'corruptness', 'costly', 'counterclaim', 'counterclaimed',
                       'counterclaiming', 'counterclaims', 'counterfeit', 'counterfeited', 'counterfeiter',
                       'counterfeiters', 'counterfeiting', 'counterfeits', 'countermeasure', 'countermeasures', 'crime',
                       'crimes', 'criminal', 'criminally', 'criminals', 'crises', 'crisis', 'critical', 'critically',
                       'criticism', 'criticisms', 'criticize', 'criticized', 'criticizes', 'criticizing', 'crucial',
                       'crucially', 'culpability', 'culpable', 'culpably', 'cumbersome', 'curtail', 'curtailed',
                       'curtailing', 'curtailment', 'curtailments', 'curtails', 'cut', 'cutback', 'cutbacks',
                       'cyberattack', 'cyberattacks', 'cyberbullying', 'cybercrime', 'cybercrimes', 'cybercriminal',
                       'cybercriminals', 'damage', 'damaged', 'damages', 'damaging', 'dampen', 'dampened', 'danger',
                       'dangerous', 'dangerously', 'dangers', 'deadlock', 'deadlocked', 'deadlocking', 'deadlocks',
                       'deadweight', 'deadweights', 'debarment', 'debarments', 'debarred', 'deceased', 'deceit',
                       'deceitful', 'deceitfulness', 'deceive', 'deceived', 'deceives', 'deceiving', 'deception',
                       'deceptions', 'deceptive', 'deceptively', 'decline', 'declined', 'declines', 'declining',
                       'deface', 'defaced', 'defacement', 'defamation', 'defamations', 'defamatory', 'defame',
                       'defamed', 'defames', 'defaming', 'default', 'defaulted', 'defaulting', 'defaults', 'defeat',
                       'defeated', 'defeating', 'defeats', 'defect', 'defective', 'defects', 'defend', 'defendant',
                       'defendants', 'defended', 'defending', 'defends', 'defensive', 'defer', 'deficiencies',
                       'deficiency', 'deficient', 'deficit', 'deficits', 'defraud', 'defrauded', 'defrauding',
                       'defrauds', 'defunct', 'degradation', 'degradations', 'degrade', 'degraded', 'degrades',
                       'degrading', 'delay', 'delayed', 'delaying', 'delays', 'deleterious', 'deliberate',
                       'deliberated', 'deliberately', 'delinquencies', 'delinquency', 'delinquent', 'delinquently',
                       'delinquents', 'delist', 'delisted', 'delisting', 'delists', 'demise', 'demised', 'demises',
                       'demising', 'demolish', 'demolished', 'demolishes', 'demolishing', 'demolition', 'demolitions',
                       'demote', 'demoted', 'demotes', 'demoting', 'demotion', 'demotions', 'denial', 'denials',
                       'denied', 'denies', 'denigrate', 'denigrated', 'denigrates', 'denigrating', 'denigration',
                       'deny', 'denying', 'deplete', 'depleted', 'depletes', 'depleting', 'depletion', 'depletions',
                       'deprecation', 'depress', 'depressed', 'depresses', 'depressing', 'deprivation', 'deprive',
                       'deprived', 'deprives', 'depriving', 'derelict', 'dereliction', 'derogatory', 'destabilization',
                       'destabilize', 'destabilized', 'destabilizing', 'destroy', 'destroyed', 'destroying', 'destroys',
                       'destruction', 'destructive', 'detain', 'detained', 'detention', 'detentions', 'deter',
                       'deteriorate', 'deteriorated', 'deteriorates', 'deteriorating', 'deterioration',
                       'deteriorations', 'deterred', 'deterrence', 'deterrences', 'deterrent', 'deterrents',
                       'deterring', 'deters', 'detract', 'detracted', 'detracting', 'detriment', 'detrimental',
                       'detrimentally', 'detriments', 'devalue', 'devalued', 'devalues', 'devaluing', 'devastate',
                       'devastated', 'devastating', 'devastation', 'deviate', 'deviated', 'deviates', 'deviating',
                       'deviation', 'deviations', 'devolve', 'devolved', 'devolves', 'devolving', 'difficult',
                       'difficulties', 'difficultly', 'difficulty', 'diminish', 'diminished', 'diminishes',
                       'diminishing', 'diminution', 'disadvantage', 'disadvantaged', 'disadvantageous', 'disadvantages',
                       'disaffiliation', 'disagree', 'disagreeable', 'disagreed', 'disagreeing', 'disagreement',
                       'disagreements', 'disagrees', 'disallow', 'disallowance', 'disallowances', 'disallowed',
                       'disallowing', 'disallows', 'disappear', 'disappearance', 'disappearances', 'disappeared',
                       'disappearing', 'disappears', 'disappoint', 'disappointed', 'disappointing', 'disappointingly',
                       'disappointment', 'disappointments', 'disappoints', 'disapproval', 'disapprovals', 'disapprove',
                       'disapproved', 'disapproves', 'disapproving', 'disassociates', 'disassociating',
                       'disassociation', 'disassociations', 'disaster', 'disasters', 'disastrous', 'disastrously',
                       'disavow', 'disavowal', 'disavowed', 'disavowing', 'disavows', 'disciplinary', 'disclaim',
                       'disclaimed', 'disclaimer', 'disclaimers', 'disclaiming', 'disclaims', 'disclose', 'disclosed',
                       'discloses', 'disclosing', 'discontinuance', 'discontinuances', 'discontinuation',
                       'discontinuations', 'discontinue', 'discontinued', 'discontinues', 'discontinuing', 'discourage',
                       'discouraged', 'discourages', 'discouraging', 'discredit', 'discredited', 'discrediting',
                       'discredits', 'discrepancies', 'discrepancy', 'disfavor', 'disfavored', 'disfavoring',
                       'disfavors', 'disgorge', 'disgorged', 'disgorgement', 'disgorgements', 'disgorges', 'disgorging',
                       'disgrace', 'disgraceful', 'disgracefully', 'dishonest', 'dishonestly', 'dishonesty', 'dishonor',
                       'dishonorable', 'dishonorably', 'dishonored', 'dishonoring', 'dishonors', 'disincentives',
                       'disinterested', 'disinterestedly', 'disinterestedness', 'disloyal', 'disloyally', 'disloyalty',
                       'dismal', 'dismally', 'dismiss', 'dismissal', 'dismissals', 'dismissed', 'dismisses',
                       'dismissing', 'disorderly', 'disparage', 'disparaged', 'disparagement', 'disparagements',
                       'disparages', 'disparaging', 'disparagingly', 'disparities', 'disparity', 'displace',
                       'displaced', 'displacement', 'displacements', 'displaces', 'displacing', 'dispose', 'dispossess',
                       'dispossessed', 'dispossesses', 'dispossessing', 'disproportion', 'disproportional',
                       'disproportionate', 'disproportionately', 'dispute', 'disputed', 'disputes', 'disputing',
                       'disqualification', 'disqualifications', 'disqualified', 'disqualifies', 'disqualify',
                       'disqualifying', 'disregard', 'disregarded', 'disregarding', 'disregards', 'disreputable',
                       'disrepute', 'disrupt', 'disrupted', 'disrupting', 'disruption', 'disruptions', 'disruptive',
                       'disrupts', 'dissatisfaction', 'dissatisfied', 'dissent', 'dissented', 'dissenter', 'dissenters',
                       'dissenting', 'dissents', 'dissident', 'dissidents', 'dissolution', 'dissolutions', 'distort',
                       'distorted', 'distorting', 'distortion', 'distortions', 'distorts', 'distract', 'distracted',
                       'distracting', 'distraction', 'distractions', 'distracts', 'distress', 'distressed', 'disturb',
                       'disturbance', 'disturbances', 'disturbed', 'disturbing', 'disturbs', 'diversion', 'divert',
                       'diverted', 'diverting', 'diverts', 'divest', 'divested', 'divesting', 'divestiture',
                       'divestitures', 'divestment', 'divestments', 'divests', 'divorce', 'divorced', 'divulge',
                       'divulged', 'divulges', 'divulging', 'doubt', 'doubted', 'doubtful', 'doubts', 'downgrade',
                       'downgraded', 'downgrades', 'downgrading', 'downsize', 'downsized', 'downsizes', 'downsizing',
                       'downsizings', 'downtime', 'downtimes', 'downturn', 'downturns', 'downward', 'downwards', 'drag',
                       'drastic', 'drastically', 'drawback', 'drawbacks', 'dropped', 'drought', 'droughts', 'duress',
                       'dysfunction', 'dysfunctional', 'dysfunctions', 'easing', 'egregious', 'egregiously', 'embargo',
                       'embargoed', 'embargoes', 'embargoing', 'embarrass', 'embarrassed', 'embarrasses',
                       'embarrassing', 'embarrassment', 'embarrassments', 'embezzle', 'embezzled', 'embezzlement',
                       'embezzlements', 'embezzler', 'embezzles', 'embezzling', 'encroach', 'encroached', 'encroaches',
                       'encroaching', 'encroachment', 'encroachments', 'encumber', 'encumbered', 'encumbering',
                       'encumbers', 'encumbrance', 'encumbrances', 'endanger', 'endangered', 'endangering',
                       'endangerment', 'endangers', 'enjoin', 'enjoined', 'enjoining', 'enjoins', 'erode', 'eroded',
                       'erodes', 'eroding', 'erosion', 'erratic', 'erratically', 'erred', 'erring', 'erroneous',
                       'erroneously', 'error', 'errors', 'errs', 'escalate', 'escalated', 'escalates', 'escalating',
                       'evade', 'evaded', 'evades', 'evading', 'evasion', 'evasions', 'evasive', 'evict', 'evicted',
                       'evicting', 'eviction', 'evictions', 'evicts', 'exacerbate', 'exacerbated', 'exacerbates',
                       'exacerbating', 'exacerbation', 'exacerbations', 'exaggerate', 'exaggerated', 'exaggerates',
                       'exaggerating', 'exaggeration', 'excessive', 'excessively', 'exculpate', 'exculpated',
                       'exculpates', 'exculpating', 'exculpation', 'exculpations', 'exculpatory', 'exonerate',
                       'exonerated', 'exonerates', 'exonerating', 'exoneration', 'exonerations', 'exploit',
                       'exploitation', 'exploitations', 'exploitative', 'exploited', 'exploiting', 'exploits', 'expose',
                       'exposed', 'exposes', 'exposing', 'expropriate', 'expropriated', 'expropriates', 'expropriating',
                       'expropriation', 'expropriations', 'expulsion', 'expulsions', 'extenuating', 'fail', 'failed',
                       'failing', 'failings', 'fails', 'failure', 'failures', 'fallout', 'false', 'falsely',
                       'falsification', 'falsifications', 'falsified', 'falsifies', 'falsify', 'falsifying', 'falsity',
                       'fatalities', 'fatality', 'fatally', 'fault', 'faulted', 'faults', 'faulty', 'fear', 'fears',
                       'felonies', 'felonious', 'felony', 'fictitious', 'fined', 'fines', 'fired', 'firing', 'flaw',
                       'flawed', 'flaws', 'forbid', 'forbidden', 'forbidding', 'forbids', 'force', 'forced', 'forcing',
                       'foreclose', 'foreclosed', 'forecloses', 'foreclosing', 'foreclosure', 'foreclosures', 'forego',
                       'foregoes', 'foregone', 'forestall', 'forestalled', 'forestalling', 'forestalls', 'forfeit',
                       'forfeited', 'forfeiting', 'forfeits', 'forfeiture', 'forfeitures', 'forgers', 'forgery',
                       'fraud', 'frauds', 'fraudulence', 'fraudulent', 'fraudulently', 'frivolous', 'frivolously',
                       'frustrate', 'frustrated', 'frustrates', 'frustrating', 'frustratingly', 'frustration',
                       'frustrations', 'fugitive', 'fugitives', 'gratuitous', 'gratuitously', 'grievance', 'grievances',
                       'grossly', 'groundless', 'guilty', 'halt', 'halted', 'hamper', 'hampered', 'hampering',
                       'hampers', 'harass', 'harassed', 'harassing', 'harassment', 'hardship', 'hardships', 'harm',
                       'harmed', 'harmful', 'harmfully', 'harming', 'harms', 'harsh', 'harsher', 'harshest', 'harshly',
                       'harshness', 'hazard', 'hazardous', 'hazards', 'hinder', 'hindered', 'hindering', 'hinders',
                       'hindrance', 'hindrances', 'hostile', 'hostility', 'hurt', 'hurting', 'idle', 'idled', 'idling',
                       'ignore', 'ignored', 'ignores', 'ignoring', 'ill', 'illegal', 'illegalities', 'illegality',
                       'illegally', 'illegible', 'illicit', 'illicitly', 'illiquid', 'illiquidity', 'imbalance',
                       'imbalances', 'immature', 'immoral', 'impair', 'impaired', 'impairing', 'impairment',
                       'impairments', 'impairs', 'impasse', 'impasses', 'impede', 'impeded', 'impedes', 'impediment',
                       'impediments', 'impeding', 'impending', 'imperative', 'imperfection', 'imperfections', 'imperil',
                       'impermissible', 'implicate', 'implicated', 'implicates', 'implicating', 'impossibility',
                       'impossible', 'impound', 'impounded', 'impounding', 'impounds', 'impracticable', 'impractical',
                       'impracticalities', 'impracticality', 'imprisonment', 'improper', 'improperly', 'improprieties',
                       'impropriety', 'imprudent', 'imprudently', 'inability', 'inaccessible', 'inaccuracies',
                       'inaccuracy', 'inaccurate', 'inaccurately', 'inaction', 'inactions', 'inactivate', 'inactivated',
                       'inactivates', 'inactivating', 'inactivation', 'inactivations', 'inactivity', 'inadequacies',
                       'inadequacy', 'inadequate', 'inadequately', 'inadvertent', 'inadvertently', 'inadvisability',
                       'inadvisable', 'inappropriate', 'inappropriately', 'inattention', 'incapable', 'incapacitated',
                       'incapacity', 'incarcerate', 'incarcerated', 'incarcerates', 'incarcerating', 'incarceration',
                       'incarcerations', 'incidence', 'incidences', 'incident', 'incidents', 'incompatibilities',
                       'incompatibility', 'incompatible', 'incompetence', 'incompetency', 'incompetent',
                       'incompetently', 'incompetents', 'incomplete', 'incompletely', 'incompleteness', 'inconclusive',
                       'inconsistencies', 'inconsistency', 'inconsistent', 'inconsistently', 'inconvenience',
                       'inconveniences', 'inconvenient', 'incorrect', 'incorrectly', 'incorrectness', 'indecency',
                       'indecent', 'indefeasible', 'indefeasibly', 'indict', 'indictable', 'indicted', 'indicting',
                       'indictment', 'indictments', 'ineffective', 'ineffectively', 'ineffectiveness', 'inefficiencies',
                       'inefficiency', 'inefficient', 'inefficiently', 'ineligibility', 'ineligible', 'inequitable',
                       'inequitably', 'inequities', 'inequity', 'inevitable', 'inexperience', 'inexperienced',
                       'inferior', 'inflicted', 'infraction', 'infractions', 'infringe', 'infringed', 'infringement',
                       'infringements', 'infringes', 'infringing', 'inhibited', 'inimical', 'injunction', 'injunctions',
                       'injure', 'injured', 'injures', 'injuries', 'injuring', 'injurious', 'injury', 'inordinate',
                       'inordinately', 'inquiry', 'insecure', 'insensitive', 'insolvencies', 'insolvency', 'insolvent',
                       'instability', 'insubordination', 'insufficiency', 'insufficient', 'insufficiently',
                       'insurrection', 'insurrections', 'intentional', 'interfere', 'interfered', 'interference',
                       'interferences', 'interferes', 'interfering', 'intermittent', 'intermittently', 'interrupt',
                       'interrupted', 'interrupting', 'interruption', 'interruptions', 'interrupts', 'intimidation',
                       'intrusion', 'invalid', 'invalidate', 'invalidated', 'invalidates', 'invalidating',
                       'invalidation', 'invalidity', 'investigate', 'investigated', 'investigates', 'investigating',
                       'investigation', 'investigations', 'involuntarily', 'involuntary', 'irreconcilable',
                       'irreconcilably', 'irrecoverable', 'irrecoverably', 'irregular', 'irregularities',
                       'irregularity', 'irregularly', 'irreparable', 'irreparably', 'irreversible', 'jeopardize',
                       'jeopardized', 'justifiable', 'kickback', 'kickbacks', 'knowingly', 'lack', 'lacked', 'lacking',
                       'lackluster', 'lacks', 'lag', 'lagged', 'lagging', 'lags', 'lapse', 'lapsed', 'lapses',
                       'lapsing', 'late', 'laundering', 'layoff', 'layoffs', 'lie', 'limitation', 'limitations',
                       'lingering', 'liquidate', 'liquidated', 'liquidates', 'liquidating', 'liquidation',
                       'liquidations', 'liquidator', 'liquidators', 'litigant', 'litigants', 'litigate', 'litigated',
                       'litigates', 'litigating', 'litigation', 'litigations', 'lockout', 'lockouts', 'lose', 'loses',
                       'losing', 'loss', 'losses', 'lost', 'lying', 'malfeasance', 'malfunction', 'malfunctioned',
                       'malfunctioning', 'malfunctions', 'malice', 'malicious', 'maliciously', 'malpractice',
                       'manipulate', 'manipulated', 'manipulates', 'manipulating', 'manipulation', 'manipulations',
                       'manipulative', 'markdown', 'markdowns', 'misapplication', 'misapplications', 'misapplied',
                       'misapplies', 'misapply', 'misapplying', 'misappropriate', 'misappropriated', 'misappropriates',
                       'misappropriating', 'misappropriation', 'misappropriations', 'misbranded', 'miscalculate',
                       'miscalculated', 'miscalculates', 'miscalculating', 'miscalculation', 'miscalculations',
                       'mischaracterization', 'mischief', 'misclassification', 'misclassifications', 'misclassified',
                       'misclassify', 'miscommunication', 'misconduct', 'misdated', 'misdemeanor', 'misdemeanors',
                       'misdirected', 'mishandle', 'mishandled', 'mishandles', 'mishandling', 'misinform',
                       'misinformation', 'misinformed', 'misinforming', 'misinforms', 'misinterpret',
                       'misinterpretation', 'misinterpretations', 'misinterpreted', 'misinterpreting', 'misinterprets',
                       'misjudge', 'misjudged', 'misjudges', 'misjudging', 'misjudgment', 'misjudgments', 'mislabel',
                       'mislabeled', 'mislabeling', 'mislabelled', 'mislabels', 'mislead', 'misleading', 'misleadingly',
                       'misleads', 'misled', 'mismanage', 'mismanaged', 'mismanagement', 'mismanages', 'mismanaging',
                       'mismatch', 'mismatched', 'mismatches', 'mismatching', 'misplaced', 'misprice', 'mispricing',
                       'mispricings', 'misrepresent', 'misrepresentation', 'misrepresentations', 'misrepresented',
                       'misrepresenting', 'misrepresents', 'miss', 'missed', 'misses', 'misstate', 'misstated',
                       'misstatement', 'misstatements', 'misstates', 'misstating', 'misstep', 'missteps', 'mistake',
                       'mistaken', 'mistakenly', 'mistakes', 'mistaking', 'mistrial', 'mistrials', 'misunderstand',
                       'misunderstanding', 'misunderstandings', 'misunderstood', 'misuse', 'misused', 'misuses',
                       'misusing', 'monopolistic', 'monopolists', 'monopolization', 'monopolize', 'monopolized',
                       'monopolizes', 'monopolizing', 'monopoly', 'moratoria', 'moratorium', 'moratoriums',
                       'mothballed', 'mothballing', 'negative', 'negatively', 'negatives', 'neglect', 'neglected',
                       'neglectful', 'neglecting', 'neglects', 'negligence', 'negligences', 'negligent', 'negligently',
                       'nonattainment', 'noncompetitive', 'noncompliance', 'noncompliances', 'noncompliant',
                       'noncomplying', 'nonconforming', 'nonconformities', 'nonconformity', 'nondisclosure',
                       'nonfunctional', 'nonpayment', 'nonpayments', 'nonperformance', 'nonperformances',
                       'nonperforming', 'nonproducing', 'nonproductive', 'nonrecoverable', 'nonrenewal', 'nuisance',
                       'nuisances', 'nullification', 'nullifications', 'nullified', 'nullifies', 'nullify',
                       'nullifying', 'objected', 'objecting', 'objection', 'objectionable', 'objectionably',
                       'objections', 'obscene', 'obscenity', 'obsolescence', 'obsolete', 'obstacle', 'obstacles',
                       'obstruct', 'obstructed', 'obstructing', 'obstruction', 'obstructions', 'offence', 'offences',
                       'offend', 'offended', 'offender', 'offenders', 'offending', 'offends', 'omission', 'omissions',
                       'omit', 'omits', 'omitted', 'omitting', 'onerous', 'opportunistic', 'opportunistically',
                       'oppose', 'opposed', 'opposes', 'opposing', 'opposition', 'oppositions', 'outage', 'outages',
                       'outdated', 'outmoded', 'overage', 'overages', 'overbuild', 'overbuilding', 'overbuilds',
                       'overbuilt', 'overburden', 'overburdened', 'overburdening', 'overcapacities', 'overcapacity',
                       'overcharge', 'overcharged', 'overcharges', 'overcharging', 'overcome', 'overcomes',
                       'overcoming', 'overdue', 'overestimate', 'overestimated', 'overestimates', 'overestimating',
                       'overestimation', 'overestimations', 'overload', 'overloaded', 'overloading', 'overloads',
                       'overlook', 'overlooked', 'overlooking', 'overlooks', 'overpaid', 'overpayment', 'overpayments',
                       'overproduced', 'overproduces', 'overproducing', 'overproduction', 'overrun', 'overrunning',
                       'overruns', 'overshadow', 'overshadowed', 'overshadowing', 'overshadows', 'overstate',
                       'overstated', 'overstatement', 'overstatements', 'overstates', 'overstating', 'oversupplied',
                       'oversupplies', 'oversupply', 'oversupplying', 'overtly', 'overturn', 'overturned',
                       'overturning', 'overturns', 'overvalue', 'overvalued', 'overvaluing', 'panic', 'panics',
                       'penalize', 'penalized', 'penalizes', 'penalizing', 'penalties', 'penalty', 'peril', 'perils',
                       'perjury', 'perpetrate', 'perpetrated', 'perpetrates', 'perpetrating', 'perpetration', 'persist',
                       'persisted', 'persistence', 'persistent', 'persistently', 'persisting', 'persists', 'pervasive',
                       'pervasively', 'pervasiveness', 'petty', 'picket', 'picketed', 'picketing', 'plaintiff',
                       'plaintiffs', 'plea', 'plead', 'pleaded', 'pleading', 'pleadings', 'pleads', 'pleas', 'pled',
                       'poor', 'poorly', 'poses', 'posing', 'postpone', 'postponed', 'postponement', 'postponements',
                       'postpones', 'postponing', 'precipitated', 'precipitous', 'precipitously', 'preclude',
                       'precluded', 'precludes', 'precluding', 'predatory', 'prejudice', 'prejudiced', 'prejudices',
                       'prejudicial', 'prejudicing', 'premature', 'prematurely', 'pressing', 'pretrial', 'preventing',
                       'prevention', 'prevents', 'problem', 'problematic', 'problematical', 'problems', 'prolong',
                       'prolongation', 'prolongations', 'prolonged', 'prolonging', 'prolongs', 'prone', 'prosecute',
                       'prosecuted', 'prosecutes', 'prosecuting', 'prosecution', 'prosecutions', 'protest', 'protested',
                       'protester', 'protesters', 'protesting', 'protestor', 'protestors', 'protests', 'protracted',
                       'protraction', 'provoke', 'provoked', 'provokes', 'provoking', 'punished', 'punishes',
                       'punishing', 'punishment', 'punishments', 'punitive', 'purport', 'purported', 'purportedly',
                       'purporting', 'purports', 'question', 'questionable', 'questionably', 'questioned',
                       'questioning', 'questions', 'quit', 'quitting', 'racketeer', 'racketeering', 'rationalization',
                       'rationalizations', 'rationalize', 'rationalized', 'rationalizes', 'rationalizing',
                       'reassessment', 'reassessments', 'reassign', 'reassigned', 'reassigning', 'reassignment',
                       'reassignments', 'reassigns', 'recall', 'recalled', 'recalling', 'recalls', 'recession',
                       'recessionary', 'recessions', 'reckless', 'recklessly', 'recklessness', 'redact', 'redacted',
                       'redacting', 'redaction', 'redactions', 'redefault', 'redefaulted', 'redefaults', 'redress',
                       'redressed', 'redresses', 'redressing', 'refusal', 'refusals', 'refuse', 'refused', 'refuses',
                       'refusing', 'reject', 'rejected', 'rejecting', 'rejection', 'rejections', 'rejects',
                       'relinquish', 'relinquished', 'relinquishes', 'relinquishing', 'relinquishment',
                       'relinquishments', 'reluctance', 'reluctant', 'renegotiate', 'renegotiated', 'renegotiates',
                       'renegotiating', 'renegotiation', 'renegotiations', 'renounce', 'renounced', 'renouncement',
                       'renouncements', 'renounces', 'renouncing', 'reparation', 'reparations', 'repossessed',
                       'repossesses', 'repossessing', 'repossession', 'repossessions', 'repudiate', 'repudiated',
                       'repudiates', 'repudiating', 'repudiation', 'repudiations', 'resign', 'resignation',
                       'resignations', 'resigned', 'resigning', 'resigns', 'restate', 'restated', 'restatement',
                       'restatements', 'restates', 'restating', 'restructure', 'restructured', 'restructures',
                       'restructuring', 'restructurings', 'retaliate', 'retaliated', 'retaliates', 'retaliating',
                       'retaliation', 'retaliations', 'retaliatory', 'retribution', 'retributions', 'revocation',
                       'revocations', 'revoke', 'revoked', 'revokes', 'revoking', 'ridicule', 'ridiculed', 'ridicules',
                       'ridiculing', 'riskier', 'riskiest', 'risky', 'sabotage', 'sacrifice', 'sacrificed',
                       'sacrifices', 'sacrificial', 'sacrificing', 'scandalous', 'scandals', 'scrutinize',
                       'scrutinized', 'scrutinizes', 'scrutinizing', 'scrutiny', 'secrecy', 'seize', 'seized', 'seizes',
                       'seizing', 'sentenced', 'sentencing', 'serious', 'seriously', 'seriousness', 'setback',
                       'setbacks', 'sever', 'severe', 'severed', 'severely', 'severities', 'severity', 'sharply',
                       'shocked', 'shortage', 'shortages', 'shortfall', 'shortfalls', 'shrinkage', 'shrinkages', 'shut',
                       'shutdown', 'shutdowns', 'shuts', 'shutting', 'slander', 'slandered', 'slanderous', 'slanders',
                       'slippage', 'slippages', 'slow', 'slowdown', 'slowdowns', 'slowed', 'slower', 'slowest',
                       'slowing', 'slowly', 'slowness', 'sluggish', 'sluggishly', 'sluggishness', 'solvencies',
                       'solvency', 'spam', 'spammers', 'spamming', 'staggering', 'stagnant', 'stagnate', 'stagnated',
                       'stagnates', 'stagnating', 'stagnation', 'standstill', 'standstills', 'stolen', 'stoppage',
                       'stoppages', 'stopped', 'stopping', 'stops', 'strain', 'strained', 'straining', 'strains',
                       'stress', 'stressed', 'stresses', 'stressful', 'stressing', 'stringent', 'subjected',
                       'subjecting', 'subjection', 'subpoena', 'subpoenaed', 'subpoenas', 'substandard', 'sue', 'sued',
                       'sues', 'suffer', 'suffered', 'suffering', 'suffers', 'suing', 'summoned', 'summoning',
                       'summons', 'summonses', 'susceptibility', 'susceptible', 'suspect', 'suspected', 'suspects',
                       'suspend', 'suspended', 'suspending', 'suspends', 'suspension', 'suspensions', 'suspicion',
                       'suspicions', 'suspicious', 'suspiciously', 'taint', 'tainted', 'tainting', 'taints', 'tampered',
                       'tense', 'terminate', 'terminated', 'terminates', 'terminating', 'termination', 'terminations',
                       'testify', 'testifying', 'threat', 'threaten', 'threatened', 'threatening', 'threatens',
                       'threats', 'tightening', 'tolerate', 'tolerated', 'tolerates', 'tolerating', 'toleration',
                       'tortuous', 'tortuously', 'tragedies', 'tragedy', 'tragic', 'tragically', 'traumatic', 'trouble',
                       'troubled', 'troubles', 'turbulence', 'turmoil', 'unable', 'unacceptable', 'unacceptably',
                       'unaccounted', 'unannounced', 'unanticipated', 'unapproved', 'unattractive', 'unauthorized',
                       'unavailability', 'unavailable', 'unavoidable', 'unavoidably', 'unaware', 'uncollectable',
                       'uncollected', 'uncollectibility', 'uncollectible', 'uncollectibles', 'uncompetitive',
                       'uncompleted', 'unconscionable', 'unconscionably', 'uncontrollable', 'uncontrollably',
                       'uncontrolled', 'uncorrected', 'uncover', 'uncovered', 'uncovering', 'uncovers', 'undeliverable',
                       'undelivered', 'undercapitalized', 'undercut', 'undercuts', 'undercutting', 'underestimate',
                       'underestimated', 'underestimates', 'underestimating', 'underestimation', 'underfunded',
                       'underinsured', 'undermine', 'undermined', 'undermines', 'undermining', 'underpaid',
                       'underpayment', 'underpayments', 'underpays', 'underperform', 'underperformance',
                       'underperformed', 'underperforming', 'underperforms', 'underproduced', 'underproduction',
                       'underreporting', 'understate', 'understated', 'understatement', 'understatements',
                       'understates', 'understating', 'underutilization', 'underutilized', 'undesirable', 'undesired',
                       'undetected', 'undetermined', 'undisclosed', 'undocumented', 'undue', 'unduly', 'uneconomic',
                       'uneconomical', 'uneconomically', 'unemployed', 'unemployment', 'unethical', 'unethically',
                       'unexcused', 'unexpected', 'unexpectedly', 'unfair', 'unfairly', 'unfavorability', 'unfavorable',
                       'unfavorably', 'unfavourable', 'unfeasible', 'unfit', 'unfitness', 'unforeseeable', 'unforeseen',
                       'unforseen', 'unfortunate', 'unfortunately', 'unfounded', 'unfriendly', 'unfulfilled',
                       'unfunded', 'uninsured', 'unintended', 'unintentional', 'unintentionally', 'unjust',
                       'unjustifiable', 'unjustifiably', 'unjustified', 'unjustly', 'unknowing', 'unknowingly',
                       'unlawful', 'unlawfully', 'unlicensed', 'unliquidated', 'unmarketable', 'unmerchantable',
                       'unmeritorious', 'unnecessarily', 'unnecessary', 'unneeded', 'unobtainable', 'unoccupied',
                       'unpaid', 'unperformed', 'unplanned', 'unpopular', 'unpredictability', 'unpredictable',
                       'unpredictably', 'unpredicted', 'unproductive', 'unprofitability', 'unprofitable', 'unqualified',
                       'unrealistic', 'unreasonable', 'unreasonableness', 'unreasonably', 'unreceptive',
                       'unrecoverable', 'unrecovered', 'unreimbursed', 'unreliable', 'unremedied', 'unreported',
                       'unresolved', 'unrest', 'unsafe', 'unsalable', 'unsaleable', 'unsatisfactory', 'unsatisfied',
                       'unsavory', 'unscheduled', 'unsellable', 'unsold', 'unsound', 'unstabilized', 'unstable',
                       'unsubstantiated', 'unsuccessful', 'unsuccessfully', 'unsuitability', 'unsuitable', 'unsuitably',
                       'unsuited', 'unsure', 'unsuspected', 'unsuspecting', 'unsustainable', 'untenable', 'untimely',
                       'untrusted', 'untruth', 'untruthful', 'untruthfully', 'untruthfulness', 'untruths', 'unusable',
                       'unwanted', 'unwarranted', 'unwelcome', 'unwilling', 'unwillingness', 'upset', 'urgency',
                       'urgent', 'usurious', 'usurp', 'usurped', 'usurping', 'usurps', 'usury', 'vandalism', 'verdict',
                       'verdicts', 'vetoed', 'victims', 'violate', 'violated', 'violates', 'violating', 'violation',
                       'violations', 'violative', 'violator', 'violators', 'violence', 'violent', 'violently',
                       'vitiate', 'vitiated', 'vitiates', 'vitiating', 'vitiation', 'voided', 'voiding', 'volatile',
                       'volatility', 'vulnerabilities', 'vulnerability', 'vulnerable', 'vulnerably', 'warn', 'warned',
                       'warning', 'warnings', 'warns', 'wasted', 'wasteful', 'wasting', 'weak', 'weaken', 'weakened',
                       'weakening', 'weakens', 'weaker', 'weakest', 'weakly', 'weakness', 'weaknesses', 'willfully',
                       'worries', 'worry', 'worrying', 'worse', 'worsen', 'worsened', 'worsening', 'worsens', 'worst',
                       'worthless', 'writedown', 'writedowns', 'writeoff', 'writeoffs', 'wrong', 'wrongdoing',
                       'wrongdoings', 'wrongful', 'wrongfully', 'wrongly', 'negative', 'negatives', 'fail', 'fails', 'failing', 'failure', 'weak', 'weakness', 'weaknesses',
                      'difficult', 'difficulty', 'hurdle', 'hurdles', 'obstacle', 'obstacles', 'slump', 'slumps',
                      'slumping', 'slumped', 'uncertain', 'uncertainty', 'unsettled', 'unfavorable', 'downturn',
                      'depressed', 'disappoint', 'disappoints', 'disappointing', 'disappointed', 'disappointment',
                      'risk', 'risks', 'risky', 'threat', 'threats', 'penalty', 'penalties', 'down', 'decrease',
                      'decreases', 'decreasing', 'decreased', 'decline', 'declines', 'declining', 'declined', 'fall',
                      'falls', 'falling', 'fell', 'fallen', 'drop', 'drops', 'dropping', 'dropped', 'deteriorate',
                      'deteriorates', 'deteriorating', 'deteriorated', 'worsen', 'worsens', 'worsening', 'weaken',
                      'weakens', 'weakening', 'weakened', 'worse', 'worst', 'low', 'lower', 'lowest', 'less', 'least',
                      'smaller', 'smallest', 'shrink', 'shrinks', 'shrinking', 'shrunk', 'below', 'under', 'challenge',
                      'challenges', 'challenging', 'challenged'] 

In [333]:
# Introduce positive financial keywords
Positive = ['able', 'abundance', 'abundant', 'acclaimed', 'accomplish', 'accomplished', 'accomplishes',
                       'accomplishing', 'accomplishment', 'accomplishments', 'achieve', 'achieved', 'achievement',
                       'achievements', 'achieves', 'achieving', 'adequately', 'advancement', 'advancements', 'advances',
                       'advancing', 'advantage', 'advantaged', 'advantageous', 'advantageously', 'advantages',
                       'alliance', 'alliances', 'assure', 'assured', 'assures', 'assuring', 'attain', 'attained',
                       'attaining', 'attainment', 'attainments', 'attains', 'attractive', 'attractiveness', 'beautiful',
                       'beautifully', 'beneficial', 'beneficially', 'benefit', 'benefited', 'benefiting', 'benefitted',
                       'benefitting', 'best', 'better', 'bolstered', 'bolstering', 'bolsters', 'boom', 'booming',
                       'boost', 'boosted', 'breakthrough', 'breakthroughs', 'brilliant', 'charitable', 'collaborate',
                       'collaborated', 'collaborates', 'collaborating', 'collaboration', 'collaborations',
                       'collaborative', 'collaborator', 'collaborators', 'compliment', 'complimentary', 'complimented',
                       'complimenting', 'compliments', 'conclusive', 'conclusively', 'conducive', 'confident',
                       'constructive', 'constructively', 'courteous', 'creative', 'creatively', 'creativeness',
                       'creativity', 'delight', 'delighted', 'delightful', 'delightfully', 'delighting', 'delights',
                       'dependability', 'dependable', 'desirable', 'desired', 'despite', 'destined', 'diligent',
                       'diligently', 'distinction', 'distinctions', 'distinctive', 'distinctively', 'distinctiveness',
                       'dream', 'easier', 'easily', 'easy', 'effective', 'efficiencies', 'efficiency', 'efficient',
                       'efficiently', 'empower', 'empowered', 'empowering', 'empowers', 'enable', 'enabled', 'enables',
                       'enabling', 'encouraged', 'encouragement', 'encourages', 'encouraging', 'enhance', 'enhanced',
                       'enhancement', 'enhancements', 'enhances', 'enhancing', 'enjoy', 'enjoyable', 'enjoyably',
                       'enjoyed', 'enjoying', 'enjoyment', 'enjoys', 'enthusiasm', 'enthusiastic', 'enthusiastically',
                       'excellence', 'excellent', 'excelling', 'excels', 'exceptional', 'exceptionally', 'excited',
                       'excitement', 'exciting', 'exclusive', 'exclusively', 'exclusiveness', 'exclusives',
                       'exclusivity', 'exemplary', 'fantastic', 'favorable', 'favorably', 'favored', 'favoring',
                       'favorite', 'favorites', 'friendly', 'gain', 'gained', 'gaining', 'gains', 'good', 'great',
                       'greater', 'greatest', 'greatly', 'greatness', 'happiest', 'happily', 'happiness', 'happy',
                       'highest', 'honor', 'honorable', 'honored', 'honoring', 'honors', 'ideal', 'impress',
                       'impressed', 'impresses', 'impressing', 'impressive', 'impressively', 'improve', 'improved',
                       'improvement', 'improvements', 'improves', 'improving', 'incredible', 'incredibly',
                       'influential', 'informative', 'ingenuity', 'innovate', 'innovated', 'innovates', 'innovating',
                       'innovation', 'innovations', 'innovative', 'innovativeness', 'innovator', 'innovators',
                       'insightful', 'inspiration', 'inspirational', 'integrity', 'invent', 'invented', 'inventing',
                       'invention', 'inventions', 'inventive', 'inventiveness', 'inventor', 'inventors', 'leadership',
                       'leading', 'loyal', 'lucrative', 'meritorious', 'opportunities', 'opportunity', 'optimistic',
                       'outperform', 'outperformed', 'outperforming', 'outperforms', 'perfect', 'perfected',
                       'perfectly', 'perfects', 'pleasant', 'pleasantly', 'pleased', 'pleasure', 'plentiful', 'popular',
                       'popularity', 'positive', 'positively', 'preeminence', 'preeminent', 'premier', 'premiere',
                       'prestige', 'prestigious', 'proactive', 'proactively', 'proficiency', 'proficient',
                       'proficiently', 'profitability', 'profitable', 'profitably', 'progress', 'progressed',
                       'progresses', 'progressing', 'prospered', 'prospering', 'prosperity', 'prosperous', 'prospers',
                       'rebound', 'rebounded', 'rebounding', 'receptive', 'regain', 'regained', 'regaining', 'resolve',
                       'revolutionize', 'revolutionized', 'revolutionizes', 'revolutionizing', 'reward', 'rewarded',
                       'rewarding', 'rewards', 'satisfaction', 'satisfactorily', 'satisfactory', 'satisfied',
                       'satisfies', 'satisfy', 'satisfying', 'smooth', 'smoothing', 'smoothly', 'smooths', 'solves',
                       'solving', 'spectacular', 'spectacularly', 'stability', 'stabilization', 'stabilizations',
                       'stabilize', 'stabilized', 'stabilizes', 'stabilizing', 'stable', 'strength', 'strengthen',
                       'strengthened', 'strengthening', 'strengthens', 'strengths', 'strong', 'stronger', 'strongest',
                       'succeed', 'succeeded', 'succeeding', 'succeeds', 'success', 'successes', 'successful',
                       'successfully', 'superior', 'surpass', 'surpassed', 'surpasses', 'surpassing', 'transparency',
                       'tremendous', 'tremendously', 'unmatched', 'unparalleled', 'unsurpassed', 'upturn', 'upturns',
                       'valuable', 'versatile', 'versatility', 'vibrancy', 'vibrant', 'win', 'winner', 'winners',
                       'winning', 'worthy', 'positive', 'positives', 'success', 'successes', 'successful', 'succeed', 'succeeds',
                      'succeeding', 'succeeded', 'accomplish', 'accomplishes', 'accomplishing', 'accomplished',
                      'accomplishment', 'accomplishments', 'strong', 'strength', 'strengths', 'certain', 'certainty',
                      'definite', 'solid', 'excellent', 'good', 'leading', 'achieve', 'achieves', 'achieved',
                      'achieving', 'achievement', 'achievements', 'progress', 'progressing', 'deliver', 'delivers',
                      'delivered', 'delivering', 'leader', 'leading', 'pleased', 'reward', 'rewards', 'rewarding',
                      'rewarded', 'opportunity', 'opportunities', 'enjoy', 'enjoys', 'enjoying', 'enjoyed',
                      'encouraged', 'encouraging', 'up', 'increase', 'increases', 'increasing', 'increased', 'rise',
                      'rises', 'rising', 'rose', 'risen', 'improve', 'improves', 'improving', 'improved', 'improvement',
                      'improvements', 'strengthen', 'strengthens', 'strengthening', 'strengthened', 'stronger',
                      'strongest', 'better', 'best', 'more', 'most', 'above', 'record', 'high', 'higher', 'highest',
                      'greater', 'greatest', 'larger', 'largest', 'grow', 'grows', 'growing', 'grew', 'grown', 'growth',
                      'expand', 'expands', 'expanding', 'expanded', 'expansion', 'exceed', 'exceeds', 'exceeded',
                      'exceeding', 'beat', 'beats', 'beating']

In [334]:
# Sentiment analysis using the above negative and positive keywords.
sentbydaypos=defaultdict(int)
sentbydayneg=defaultdict(int)
sentbyday=defaultdict(int)
for index,row in df.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in Positive:
            sentbydaypos[row['day']] += 1
            sentbyday[row['day']] += 1
        if word in Negative:
            sentbydayneg[row['day']] += 1
            sentbyday[row['day']] =  sentbyday[row['day']] -1
            
#     break

In [335]:
# positive sentiment counts of frequency of positive keywords by day
sentbydaypos

defaultdict(int,
            {Timestamp('2021-03-18 00:00:00'): 18,
             Timestamp('2021-02-10 00:00:00'): 18,
             Timestamp('2020-10-06 00:00:00'): 68,
             Timestamp('2020-08-27 00:00:00'): 30,
             Timestamp('2020-06-19 00:00:00'): 12,
             Timestamp('2020-05-21 00:00:00'): 11,
             Timestamp('2020-05-13 00:00:00'): 10,
             Timestamp('2020-04-09 00:00:00'): 20,
             Timestamp('2019-11-25 00:00:00'): 34,
             Timestamp('2019-10-09 00:00:00'): 16,
             Timestamp('2019-10-08 00:00:00'): 60,
             Timestamp('2019-10-07 00:00:00'): 14,
             Timestamp('2019-10-04 00:00:00'): 15,
             Timestamp('2019-08-23 00:00:00'): 43,
             Timestamp('2019-07-16 00:00:00'): 49,
             Timestamp('2019-06-25 00:00:00'): 33,
             Timestamp('2019-06-04 00:00:00'): 15,
             Timestamp('2019-05-20 00:00:00'): 46,
             Timestamp('2019-05-09 00:00:00'): 28,
             T

In [336]:
# Create a dataframe of the positive sentiment counts of frequency of positive keywords by day
powell_sentbydayposdf = pd.DataFrame([sentbydaypos]).transpose().reset_index()
powell_sentbydayposdf.columns=(['Date','Pos_Count'])
powell_sentbydayposdf

,Date,Pos_Count
0,2021-03-18,18
1,2021-02-10,18
2,2020-10-06,68
3,2020-08-27,30
4,2020-06-19,12
5,2020-05-21,11
6,2020-05-13,10
7,2020-04-09,20
8,2019-11-25,34
9,2019-10-09,16


In [337]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
powell_sentbydayposdf.to_csv('./Data/powell_sentbydayposdf_count.csv', index=False)

In [338]:
# negative sentiment counts of frequency of negative keywords by day
sentbydayneg

defaultdict(int,
            {Timestamp('2021-03-18 00:00:00'): 8,
             Timestamp('2021-02-10 00:00:00'): 13,
             Timestamp('2020-10-06 00:00:00'): 76,
             Timestamp('2020-08-27 00:00:00'): 28,
             Timestamp('2020-06-19 00:00:00'): 15,
             Timestamp('2020-05-21 00:00:00'): 11,
             Timestamp('2020-05-13 00:00:00'): 24,
             Timestamp('2020-04-09 00:00:00'): 22,
             Timestamp('2019-11-25 00:00:00'): 31,
             Timestamp('2019-10-09 00:00:00'): 11,
             Timestamp('2019-10-08 00:00:00'): 36,
             Timestamp('2019-10-07 00:00:00'): 8,
             Timestamp('2019-10-04 00:00:00'): 12,
             Timestamp('2019-08-23 00:00:00'): 62,
             Timestamp('2019-07-16 00:00:00'): 55,
             Timestamp('2019-06-25 00:00:00'): 30,
             Timestamp('2019-06-04 00:00:00'): 21,
             Timestamp('2019-05-20 00:00:00'): 65,
             Timestamp('2019-05-09 00:00:00'): 13,
             Tim

In [339]:
# Create a dataframe of the negative sentiment counts of frequency of negative keywords by day
powell_sentbydaynegdf = pd.DataFrame([sentbydayneg]).transpose().reset_index()
powell_sentbydaynegdf.columns=(['Date','Neg_Count'])
powell_sentbydaynegdf

,Date,Neg_Count
0,2021-03-18,8
1,2021-02-10,13
2,2020-10-06,76
3,2020-08-27,28
4,2020-06-19,15
5,2020-05-21,11
6,2020-05-13,24
7,2020-04-09,22
8,2019-11-25,31
9,2019-10-09,11


In [340]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
powell_sentbydaynegdf.to_csv('./Data/powell_sentbydaynegdf_count.csv', index=False)

In [341]:
# (positive - negative) sentiment counts of frequency of (positive - negative) keywords by day
sentbyday

defaultdict(int,
            {Timestamp('2021-03-18 00:00:00'): 10,
             Timestamp('2021-02-10 00:00:00'): 5,
             Timestamp('2020-10-06 00:00:00'): -8,
             Timestamp('2020-08-27 00:00:00'): 2,
             Timestamp('2020-06-19 00:00:00'): -3,
             Timestamp('2020-05-21 00:00:00'): 0,
             Timestamp('2020-05-13 00:00:00'): -14,
             Timestamp('2020-04-09 00:00:00'): -2,
             Timestamp('2019-11-25 00:00:00'): 3,
             Timestamp('2019-10-09 00:00:00'): 5,
             Timestamp('2019-10-08 00:00:00'): 24,
             Timestamp('2019-10-07 00:00:00'): 6,
             Timestamp('2019-10-04 00:00:00'): 3,
             Timestamp('2019-08-23 00:00:00'): -19,
             Timestamp('2019-07-16 00:00:00'): -6,
             Timestamp('2019-06-25 00:00:00'): 3,
             Timestamp('2019-06-04 00:00:00'): -6,
             Timestamp('2019-05-20 00:00:00'): -19,
             Timestamp('2019-05-09 00:00:00'): 15,
             Timest

In [342]:
# Create a dataframe of the (positive - negative) sentiment counts of frequency of (positive - negative) keywords by day
powell_sentbydaydf = pd.DataFrame([sentbyday]).transpose().reset_index()
powell_sentbydaydf.columns=(['Date','Sent_Count'])
powell_sentbydaydf

,Date,Sent_Count
0,2021-03-18,10
1,2021-02-10,5
2,2020-10-06,-8
3,2020-08-27,2
4,2020-06-19,-3
5,2020-05-21,0
6,2020-05-13,-14
7,2020-04-09,-2
8,2019-11-25,3
9,2019-10-09,5


In [343]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
powell_sentbydaydf.to_csv('./Data/powell_sentbydaydf_count.csv', index=False)

In [344]:
# Sentiment analysis using the above negative and positive keywords.
sentbyweekpos=defaultdict(int)
sentbyweekneg=defaultdict(int)
sentbyweek=defaultdict(int)
for index,row in df.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in Positive:
            sentbyweekpos[row['week']] += 1
            sentbyweek[row['week']] += 1
        if word in Negative:
            sentbyweekneg[row['week']] += 1
            sentbyweek[row['week']] =  sentbyweek[row['week']] -1
            
#     break

In [345]:
# positive sentiment counts of frequency of positive keywords by week
sentbyweekpos

defaultdict(int,
            {Timestamp('2021-03-15 00:00:00'): 18,
             Timestamp('2021-02-08 00:00:00'): 18,
             Timestamp('2020-10-05 00:00:00'): 68,
             Timestamp('2020-08-24 00:00:00'): 30,
             Timestamp('2020-06-15 00:00:00'): 12,
             Timestamp('2020-05-18 00:00:00'): 11,
             Timestamp('2020-05-11 00:00:00'): 10,
             Timestamp('2020-04-06 00:00:00'): 20,
             Timestamp('2019-11-25 00:00:00'): 34,
             Timestamp('2019-10-07 00:00:00'): 90,
             Timestamp('2019-09-30 00:00:00'): 15,
             Timestamp('2019-08-19 00:00:00'): 43,
             Timestamp('2019-07-15 00:00:00'): 49,
             Timestamp('2019-06-24 00:00:00'): 33,
             Timestamp('2019-06-03 00:00:00'): 15,
             Timestamp('2019-05-20 00:00:00'): 46,
             Timestamp('2019-05-06 00:00:00'): 28,
             Timestamp('2019-03-11 00:00:00'): 9,
             Timestamp('2019-03-04 00:00:00'): 51,
             Ti

In [346]:
# Create a dataframe of the positive sentiment counts of frequency of positive keywords by week
powell_sentbyweekposdf = pd.DataFrame([sentbydaypos]).transpose().reset_index()
powell_sentbyweekposdf.columns=(['Week','Pos_Count'])
powell_sentbyweekposdf

,Week,Pos_Count
0,2021-03-18,18
1,2021-02-10,18
2,2020-10-06,68
3,2020-08-27,30
4,2020-06-19,12
5,2020-05-21,11
6,2020-05-13,10
7,2020-04-09,20
8,2019-11-25,34
9,2019-10-09,16


In [347]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
powell_sentbyweekposdf.to_csv('./Data/powell_sentbyweekposdf_count.csv', index=False)

In [348]:
# Negative sentiment counts of frequency of negative keywords by week
sentbyweekneg

defaultdict(int,
            {Timestamp('2021-03-15 00:00:00'): 8,
             Timestamp('2021-02-08 00:00:00'): 13,
             Timestamp('2020-10-05 00:00:00'): 76,
             Timestamp('2020-08-24 00:00:00'): 28,
             Timestamp('2020-06-15 00:00:00'): 15,
             Timestamp('2020-05-18 00:00:00'): 11,
             Timestamp('2020-05-11 00:00:00'): 24,
             Timestamp('2020-04-06 00:00:00'): 22,
             Timestamp('2019-11-25 00:00:00'): 31,
             Timestamp('2019-10-07 00:00:00'): 55,
             Timestamp('2019-09-30 00:00:00'): 12,
             Timestamp('2019-08-19 00:00:00'): 62,
             Timestamp('2019-07-15 00:00:00'): 55,
             Timestamp('2019-06-24 00:00:00'): 30,
             Timestamp('2019-06-03 00:00:00'): 21,
             Timestamp('2019-05-20 00:00:00'): 65,
             Timestamp('2019-05-06 00:00:00'): 13,
             Timestamp('2019-03-11 00:00:00'): 8,
             Timestamp('2019-03-04 00:00:00'): 24,
             Tim

In [349]:
# Create a dataframe of the positive sentiment counts of frequency of positive keywords by week
powell_sentbyweeknegdf = pd.DataFrame([sentbydaypos]).transpose().reset_index()
powell_sentbyweeknegdf.columns=(['Week','Neg_Count'])
powell_sentbyweeknegdf

,Week,Neg_Count
0,2021-03-18,18
1,2021-02-10,18
2,2020-10-06,68
3,2020-08-27,30
4,2020-06-19,12
5,2020-05-21,11
6,2020-05-13,10
7,2020-04-09,20
8,2019-11-25,34
9,2019-10-09,16


In [350]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
powell_sentbyweeknegdf.to_csv('./Data/powell_sentbyweeknegdf_count.csv', index=False)

In [351]:
# (positive - negative) sentiment counts of frequency of (positive - negative) keywords by week
sentbyweek

defaultdict(int,
            {Timestamp('2021-03-15 00:00:00'): 10,
             Timestamp('2021-02-08 00:00:00'): 5,
             Timestamp('2020-10-05 00:00:00'): -8,
             Timestamp('2020-08-24 00:00:00'): 2,
             Timestamp('2020-06-15 00:00:00'): -3,
             Timestamp('2020-05-18 00:00:00'): 0,
             Timestamp('2020-05-11 00:00:00'): -14,
             Timestamp('2020-04-06 00:00:00'): -2,
             Timestamp('2019-11-25 00:00:00'): 3,
             Timestamp('2019-10-07 00:00:00'): 35,
             Timestamp('2019-09-30 00:00:00'): 3,
             Timestamp('2019-08-19 00:00:00'): -19,
             Timestamp('2019-07-15 00:00:00'): -6,
             Timestamp('2019-06-24 00:00:00'): 3,
             Timestamp('2019-06-03 00:00:00'): -6,
             Timestamp('2019-05-20 00:00:00'): -19,
             Timestamp('2019-05-06 00:00:00'): 15,
             Timestamp('2019-03-11 00:00:00'): 1,
             Timestamp('2019-03-04 00:00:00'): 27,
             Times

In [352]:
# Create a dataframe of the (positive - negative) sentiment counts of frequency of (positive - negative) keywords by week
powell_sentbyweekdf = pd.DataFrame([sentbyweek]).transpose().reset_index()
powell_sentbyweekdf.columns=(['Week','Sent_Count'])
powell_sentbyweekdf

,Week,Sent_Count
0,2021-03-15,10
1,2021-02-08,5
2,2020-10-05,-8
3,2020-08-24,2
4,2020-06-15,-3
5,2020-05-18,0
6,2020-05-11,-14
7,2020-04-06,-2
8,2019-11-25,3
9,2019-10-07,35


In [353]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
powell_sentbyweekdf.to_csv('./Data/powell_sentbyweekdf_count.csv', index=False)